In [6]:
from tinkoff.invest import Client, InstrumentIdType, Quotation, HistoricCandle
import os
import datetime as dt 
from icecream import ic
import pandas as pd
import numpy as np

In [7]:
target_figi_in_groupe = {
    'Акции':['BBG004730N88']
    ,'Облигации':['BBG0024TRF04']
    ,'Металлы':[]
    ,'Финансы':['RUB000UTSTOM']
}

target_rate_on_groupe = {
    'Акции':0.4
    ,'Облигации':0.4
    ,'Металлы':0.2
    ,'Финансы':0
}


def get_target_rate(figi) -> pd.Series:
    """
    Надо рейты каждой группы поделить на кол-во элементов этой группы а потом нормировать к еденичной сумме
    """
    figis = pd.Series(target_figi_in_groupe)
    rate = pd.Series(target_rate_on_groupe)
    df = pd.DataFrame({'figis':figis,'groupe_rate':rate})
    df['len'] = df['figis'].apply(len)
    df['element_rate'] = (df['groupe_rate']/df['len']).replace(np.inf,0)
    df['element_rate'] = df['element_rate'] / df['element_rate'].sum()

    target_rate = df[['figis','element_rate']].explode('figis').dropna()
    target_rate = target_rate.set_index(target_rate['figis'])
    # ic(target_rate)
    
    return target_rate['element_rate']

def day_start(day: dt.datetime):
    return dt.datetime(day.year, day.month, day.day, tzinfo=day.tzinfo)

def day_end(day: dt.datetime):
    return dt.datetime(day.year, day.month, day.day, tzinfo=day.tzinfo) + dt.timedelta(days=1)# - dt.timedelta(seconds=1)

def quotation_to_float(q: Quotation):
    return float(q.units) + float(q.nano/10**9)

In [8]:
token = os.getenv('TINKOFF_API_TOKEN')
account_id = '2224375246'
with Client(token) as client:
    # ic(client.users.get_accounts())
    all_positions = []
    for position in client.operations.get_portfolio(account_id=account_id).positions:
        instrument_info = client.instruments.get_instrument_by(id_type=1,id=position.figi).instrument
        anchor_day = dt.datetime.now(dt.timezone.utc) - dt.timedelta(1)
        dates = [day_start(anchor_day),day_end(anchor_day)]
        # ic(anchor_day)
        # ic(dates)
        candles = client.market_data.get_candles(figi=position.figi,from_=dates[0],to=dates[1],interval=5,instrument_id=position.figi).candles
        # ic(candle)
        candle = [candle for candle in candles if candle.time == dates[0]]
        if len(candle) == 0:
            if position.instrument_type == 'currency':
                candle = HistoricCandle(
                    open=Quotation(1,0)
                    ,high=Quotation(1,0)
                    ,low=Quotation(1,0)
                    ,close=Quotation(1,0)
                )
            else:
                raise ValueError(f'Нет обработчика свечей для инструментов этого типа: instrument_type = {position.instrument_type}')
        else:
            candle = candle[0]
        # ic(candle)
        positions_with_info = {
            'figi': position.figi
            ,'ticker':instrument_info.ticker
            ,'instrument_type': position.instrument_type
            ,'quantity': quotation_to_float(position.quantity)
            ,'quantity_lot': quotation_to_float(position.quantity_lots)
            ,'lot': instrument_info.lot
            ,'currency': instrument_info.currency
            ,'name': instrument_info.name
            ,'class_code': instrument_info.class_code
            ,'exchange': instrument_info.exchange
            ,'price': quotation_to_float(candle.close)
        }
        all_positions.append(positions_with_info)


In [ ]:
ap = pd.DataFrame(all_positions)
ap = ap.set_index(ap['figi'])
# ic(ap)
# ic(ap[['quantity_lot','lot','price','quantity']])
full_price = ap['quantity'] * ap['price']
# ic(full_price)
total = full_price.sum() 
# ic(total)
rate = full_price/total
# ic(rate)
figi = ap['figi']
target_rate = get_target_rate(figi)

delta_rate = target_rate - rate
# ic(delta_rate)
delta_price = delta_rate * full_price
# ic(delta_price) 
delta_quantity = delta_price / ap['price']
# ic(delta_quantity) 
delta_lot = (delta_quantity / ap['lot']).round()
delta_lot.name = 'lot_changes'
# ic(delta_lot)
result = ap.merge(delta_lot,left_index=True,right_index=True)[['figi','ticker','name','lot_changes']]
ic(result)

ic| delta_rate: BBG0024TRF04    0.293963
                BBG004730N88    0.145084
                RUB000UTSTOM   -0.439047
                dtype: float64
ic| result:                       figi        ticker              name  lot_changes
            RUB000UTSTOM  RUB000UTSTOM  RUB000UTSTOM  Российский рубль      -1735.0
            BBG004730N88  BBG004730N88          SBER         Сбер Банк          0.0
            BBG0024TRF04  BBG0024TRF04  SU46023RMFS6         ОФЗ 46023          6.0


,figi,ticker,name,lot_changes
RUB000UTSTOM,RUB000UTSTOM,RUB000UTSTOM,Российский рубль,-1735.0
BBG004730N88,BBG004730N88,SBER,Сбер Банк,0.0
BBG0024TRF04,BBG0024TRF04,SU46023RMFS6,ОФЗ 46023,6.0
